# javascript 스크래핑
- 폼을 새로 고치지 않은 상태에서의 정보 전송도 가능함 (ajax)
- js 는 함수를 변수처럼 사용할 수 있다는 좋은 기능이있다. (내가 제일 좋아하는 기능) 

## 

## 널리 쓰이는 js 라이브러리
### jQuery
- 대놓고 <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script> 를 넣어서 사용하는 것이 일반적이다.
- js가 실행된 다음에 동적으로 HTML 콘텐츠 생성할 수도 있음. 
- 즉, 여지껏 사용하던 방법으로 스크랩하면 JS가 생성한 콘텐츠는 놓치는거임.! (Ajax와 DOm에 대한 이해가 필요합니다)
- 보통 애니메이션, 대화형 콘텐츠 미디어 파일이 있어서 스크랩이 어렵습니다.

### Google 애널리틱스
- 웹사이트 방문자 추적을 위한 서비스 (50% 이상의 웹사이트가 사용하고 있음)
- 소스 코드 마지막에 <!-- Google Analytics --> 를 넣어서 사용함.
- 이 스크립트는 페이지에서 페이지로 이동하는 당신의 움직임을 추적하는 특수한 쿠키를 사용한다.

### Google 맵스
- Google의 reverse GeoCoding API 를 사용하면, 위도와 경도를 주소로 변환할 수 있음.

# AJax 와 DHTML
- 웹페이지를 새로 고치지 않고도 정보를 전송할 수 있음.
원래라면 HTTP요청을 보낸것인데요, 만약 페이지를 새로 고치지 않고 폼을 전송하거나, 서버에서 정보를 가져온 경우는 대부분 AJAX를 사용한거다.
- AJAX는 언어가 아니라 기술 묶음이다. (asynchronous JavaScript and XML) 서버에 별도 요청하지않고 정보를 주고 받기 위해 사용된다. 


# Selenium 사용 
- 웹사이트를 브라우저를 통해 제어할 수 있음.
- 목적은 웹사이트 테스트 였지만, 현재는 웹 스크레핑 도구로 자주 사용됨.
- 웹 드라이버 위에서 호출되는 API 입니다.
## 팬텀 JS를 통해 인터페이스가 없는 헤드리스 브라우저를 사용했었음 ( 메모리에 불러와서 사용은 해도, 그래픽은 전혀 렌더링 하지 않음 )
## 그러나 현재는 Chrome도 헤드리스 옵션을 지원함예따라 팬텀 JS 개발 중단됐음.

In [5]:
# 예시 페이지는 2초뒤에 로딩되는 페이지임. (3초 기다리고 스크랩하자)
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import chromedriver_autoinstaller
chromedriver_autoinstaller.install(True) # 강제로 최신 버전을 설치하지 않도록 설정

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
url = 'http://pythonscraping.com/pages/javascript/ajaxDemo.html'
driver.get(url)
time.sleep(3)
print(driver.find_element(By.ID,'content').text)
driver.close()


Here is some important text you want to retrieve!
A button to click!


## 근데 이거 계속 하다보면 네트워크 때문에 3초가 정확히 3초가 아니게 될 수 있고, 에러 날 수 있음
따라서, 요소가 나올때까지 계속 확인하다가 존재할 때만 데이터를 가져온다. 
- WebDriverWait 를 사용하면 됨 (묵시적 대기 라는 기능이다.)


### 묵시적 대기? 명시적 대기?? 
명시적 대기는 3초간 기다리는거다. 
- 묵시적 대기는 우리가기다릴 DOM의 상태는 expected_condition으로 정의한다. EC라고 썻음

In [8]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install(True) # 강제로 최신 버전을 설치하지 않도록 설정

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
url = 'http://pythonscraping.com/pages/javascript/ajaxDemo.html'

driver.get(url)

try: 
    element = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,'loadedButton')))
finally:
    print(driver.find_element(By.ID,'content').text)
    driver.close()


Here is some important text you want to retrieve!
A button to click!


# 클라이언트 페이지 리다이렉트
- 2초있다가 다른 페이지로 리다이렉트 되는 페이지임(예시)
- 페이지를 처음 불러올 때 있었던 DOM 요소 하나를 주시하고 있어야한다. 

In [6]:
import time
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install(True) # 강제로 최신 버전을 설치하지 않도록 설정


def waitForLoad(driver):
    elem = driver.find_element(By.TAG_NAME,'html')
    count = 0
    while True:
        count += 1
        if count > 20:
            print('Timeing out after 1 0 sec and returning')
            return 
        time.sleep(.5)
        try:
            elem == driver.find_element(By.TAG_NAME,'html')
        except StaleElementReferenceException: # 페이지가 리다이렉트 되면서 DOM이 바뀌었을 때
            return

#무한 반복문으로 돌리면서, 페이지가 리다이렉트 되면서 DOM이 바뀌었을 때, StaleElementReferenceException이 발생하게 되고, 이때 return을 하게 되는 것임.

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
url = 'http://pythonscraping.com/pages/javascript/redirectDemo1.html' # Demo1 -> Demo2 (redirect)
driver.get(url)
waitForLoad(driver) 
print(driver.page_source)


Timeing out after 1 0 sec and returning
<html><head>
<title>The Destination Page!</title>

</head>
<body>
This is the page you are looking for!

</body></html>


# JSON 으로 받은 데이터 파싱 연습

In [13]:
import json
from urllib.request import urlopen

def getCountry(ipAddress):
    url = 'http://api.ipstack.com/' + ipAddress
    url += 'access_key=ACCESS_KEY&amp;format=1'
    response = urlopen(url).read().decode('utf-8')
    responseJson=json.loads(response)
    return responseJson.get('country_code')

print(getCountry('50.78.253.58'))
# accesskey 없으니. 에러남.

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [19]:
# json 객체는 딕셔너리로, json 배열은 리스트, json 문자열은 문자열로 변환한다. (타언어 대비 매우 효율적인 방법임)

# json 문자열 예시 
import json
jsonString = """{"arrayOfNums":[{"number":0},{"number":1},{"number":2}],
"arrayOfFruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]}"""

jsonObj = json.loads(jsonString)

print(jsonObj.get('arrayOfNums'))
print(jsonObj.get('arrayOfNums')[1])
print(jsonObj.get('arrayOfNums')[1].get('number'))


[{'number': 0}, {'number': 1}, {'number': 2}]
{'number': 1}
1


# 문서화되지 않은 API를 찾자.
## 어쨌든 개발자가 코딩한거라서 다 API가 있다. 문서화하지 않더라도!
### 개발자모드로 수작업으로 찾기.
- API 호출은 보통 JSON, XML 이다. 요청 리스트를 필터링하라
- GET 요청에서는 URL에 넘겨받은 매겨변수가 포함된다. ( page ID 같은거) 
- 보통 API 호출은 XHR 타입이다. (근데 이거 봐도 모르겠다. ㅋ)

# 문서화 되지 않은걸 내가 문서화 해보자
## 문서화 구조 
1. HTTP 메서드
2. 입력 
- 경로 매개변수
- 헤더(쿠키 포함) 
- 본문 콘텐츠(PUT,POST)
3. 출력
- 응답헤더 
- 응답 본문 타입
- 응답 본문 필드

# API 자동으로 찾고 문서화하기. ?? 
https://github.com/REMitchell/apiscraper
이거 의존성 문제로 안 됨. (2018년도에 만들어진거라서 그럼. java 버전 문제 같기도) 